**Safe LA: Enhancing Traffic Safety Through Data-Driven Insights**

**Group Members:**  
Alexandra Gladkova  
Christina Saju  
Hao Lun Rong  
Kushwant Sai Kolli  
Ladan Asempour

In [ ]:
#Cell for all imports
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
#Load data
df_old = pd.read_csv('Traffic_Collision_Data_from_2010_to_Present.csv')

df_old.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)"
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)"
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)"
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)"
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)"


In [ ]:
#Filtering the occurence date to just 2023 and 2024
df = df_old[df_old['Date Occurred'].str.contains('2023|2024', na=False)]

print(df.shape)

(33062, 18)


Some of the columns are anonymized or are redundant with an existing column and will have no impact on our analysis, which we will be removing immediately. This includes 'DR Number', 'Date Reported', 'Area ID', 'Reporting District', 'Crime Code', 'Crime Code Description', and 'Premise Code'.

In [ ]:
#Dropping previously mentioned columns

df = df.drop(columns=['DR Number', 'Date Reported', 'Area ID', 'Reporting District', 'Crime Code', 'Crime Code Description', 'Premise Code'])

df.head()


,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,Location
846,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26.0,M,H,STREET,WASHINGTON,OLIVE,"(34.0325, -118.2676)"
19678,03/10/2024,1900,Hollenbeck,0605 3030 4024,31.0,M,C,STREET,200 W AVENUE 26,NaN,"(34.0786, -118.2174)"
44979,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31.0,M,B,STREET,SAWTELLE BL,OLYMPIC BL,"(34.0382, -118.4412)"
45032,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,SIDEWALK,ADAMS,GRAND,"(34.0265, -118.2723)"
45312,08/05/2024,835,77th Street,NaN,30.0,M,H,STREET,FLORENCE,GRAND,"(33.9747, -118.2803)"


Additionally, we will be checking data types of the columns we will be using to ensure they are in the right format, if not we will change them.

In [ ]:
#Checking the data type of each column
data_types = df.dtypes
data_types

,0
Date Occurred,object
Time Occurred,int64
Area Name,object
MO Codes,object
Victim Age,float64
Victim Sex,object
Victim Descent,object
Premise Description,object
Address,object
Cross Street,object


In [ ]:
# The victim age column should be an integer as opposed to a float, we will change the data type of that column
# Date occured and time occured will be changed in a later section
df['Victim Age'] = pd.to_numeric(df['Victim Age'], errors='coerce').astype('Int64')

The MO codes each have a definition to them, we will first seperate them by the spaces in the string, and then try to remap them into their definitions.

In [ ]:
# create new columns with a seperate mo code in each
mo_columns = df["MO Codes"].astype(str).str.split(expand=True)

mo_columns

,0,1,2,3,4,5,6,7,8,9
846,3401,3701,3036,0605,3003,3024,3033,3035,3101,4024
19678,0605,3030,4024,None,None,None,None,None,None,None
44979,3004,3028,3033,3034,3037,3030,None,None,None,None
45032,0605,3011,3028,3033,3034,3036,3101,4013,3401,3701
45312,nan,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
619235,3004,3028,3030,3101,4015,None,None,None,None,None
619236,3004,3028,4026,3037,3101,None,None,None,None,None
619237,3004,3026,3037,3038,4027,3035,3101,3401,3701,None
619238,3101,3011,3028,4026,3030,3037,None,None,None,None


In [ ]:
mo_columns.to_csv('mo_code_columns.csv')

In [ ]:
# Converts the table into a single entry
mo_codes = mo_columns.values.ravel()

In [ ]:
# only gets unique values
unique_mo_codes = pd.unique(mo_codes)

In [ ]:
#print the number of mo codes that show up
n_unique = len(unique_mo_codes)

print(n_unique)

151


In [ ]:
# prints list
print(unique_mo_codes)

['3401' '3701' '3036' '0605' '3003' '3024' '3033' '3035' '3101' '4024'
 '3030' None '3004' '3028' '3034' '3037' '3011' '4013' 'nan' '3026' '3029'
 '3025' '3039' '2004' '0910' '3008' '3006' '4009' '4002' '4025' '0947'
 '3002' '3602' '3032' '4003' '3038' '3102' '4004' '4005' '1402' '3009'
 '4027' '4026' '4010' '3014' '4011' '4012' '4001' '4014' '4015' '4017'
 '4016' '1218' '3013' '3027' '4019' '4021' '3103' '3064' '4008' '4007'
 '3104' '3012' '4018' '4020' '4006' '3001' '3040' '3020' '3007' '3010'
 '1407' '1419' '3005' '0906' '1816' '3601' '3501' '3063' '3062' '1822'
 '1609' '1309' '3016' '0604' '3018' '3015' '2018' '1300' '0416' '1312'
 '2029' '0358' '3022' '2028' '0546' '0907' '3023' '1025' '2021' '2002'
 '0918' '1501' '1004' '1303' '0601' '3017' '2101' '2042' '3019' '2024'
 '1217' '0342' '1409' '0424' '0336' '2038' '0216' '0325' '0329' '3603'
 '0443' '0913' '0422' '0432' '0352' '1420' '2303' '3021' '0400' '0328'
 '1313' '0425' '1026' '3801' '2030' '0401' '1021' '0946' '1403' '0919'
 '

In [ ]:
MO_Codes_Ref = pd.read_csv('MO_Codes_Ref.csv')

MO_Codes_Ref

,MO Code,Description
0,100,Suspect Impersonate
1,101,Aid victim
2,102,Blind
3,103,Crippled
4,104,Customer
...,...,...
772,4021,T/C - Topanga
773,4024,T/C - Central Traffic (CTD)
774,4025,T/C - South Traffic (STD)
775,4026,T/C - Valley Traffic (VTD)


In [ ]:
unique_mo_df = pd.DataFrame(unique_mo_codes, columns=['mo_code'])

# Display the DataFrame
print(unique_mo_df)

    mo_code
0      3401
1      3701
2      3036
3      0605
4      3003
..      ...
146    1903
147    1605
148    0505
149    2014
150    0417

[151 rows x 1 columns]


In [ ]:
# Creating a new SQLite database
conn = sqlite3.connect('temp_database.db')

# Create a cursor object
cursor = conn.cursor()

In [ ]:
# Convert the unique_mo_df and MO_Codes_Ref into SQL Tables
unique_mo_df.to_sql('UMO', conn, if_exists='replace', index=False)
MO_Codes_Ref.to_sql('RMO', conn, if_exists='replace', index=False)


777

In [ ]:
#join the tables to get description for each unique MO

#write the query to be executed
query = """
SELECT t1.*, t2.Description
FROM UMO t1 LEFT JOIN RMO t2
ON t1.mo_code = t2.[MO Code];
"""

# query result as a pd dataframe
df_result = pd.read_sql_query(query, conn)

# Close connection to sqlite database that we created
conn.close() #rerun the connection query if need to run this section of code again

In [ ]:
print(df_result)

    mo_code                                        Description
0      3401                            T/C - Type of Collision
1      3701                 T/C - Movement Preceding Collision
2      3036                        T/C - At Intersection - Yes
3      0605          Traffic Accident/Traffic related incident
4      3003                                   T/C - Veh vs Ped
..      ...                                                ...
146    1903  Cyberstalking (Stalking using internet to comm...
147    1605                                 Lock slip/key/pick
148    0505                           Forced to fondle suspect
149    2014                       Suspect was Uber/Lyft driver
150    0417                                             Kicked

[151 rows x 2 columns]


In [ ]:
# get rows where mo_code is null
empties = df_result[df_result['mo_code'].isna()]

print(empties)

   mo_code Description
11    None        None


In [ ]:
# Drop the rows which are null
df_result = df_result.drop([11], axis=0).reset_index(drop=True)

# String matching MO Codes

In [ ]:
MO_Code_Desc = df_result

print(MO_Code_Desc)

    mo_code                                        Description
0      3401                            T/C - Type of Collision
1      3701                 T/C - Movement Preceding Collision
2      3036                        T/C - At Intersection - Yes
3      0605          Traffic Accident/Traffic related incident
4      3003                                   T/C - Veh vs Ped
..      ...                                                ...
145    1903  Cyberstalking (Stalking using internet to comm...
146    1605                                 Lock slip/key/pick
147    0505                           Forced to fondle suspect
148    2014                       Suspect was Uber/Lyft driver
149    0417                                             Kicked

[150 rows x 2 columns]


In [ ]:
# map the mo codes and the descriptions
map = dict(zip(MO_Code_Desc['mo_code'], MO_Code_Desc['Description']))

# replace all values in the mo_code tables
mo_columns = mo_columns.replace(map)

mo_columns

,0,1,2,3,4,5,6,7,8,9
846,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD)
19678,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None
44979,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None
45032,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision
45312,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
619235,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Hit and Run Misd,T/C - PCF (A) In the Narrative,T/C - North Hollywood,None,None,None,None,None
619236,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - At Intersection - No,T/C - PCF (A) In the Narrative,None,None,None,None,None
619237,T/C - Veh vs Veh,T/C - (C) Complaint of Injury,T/C - At Intersection - No,T/C - DUI Felony,T/C - West Traffic (WTD),T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Type of Collision,T/C - Movement Preceding Collision,None
619238,T/C - PCF (A) In the Narrative,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - Hit and Run Misd,T/C - At Intersection - No,None,None,None,None


In [ ]:
# Merge the two tables
df = pd.concat([df, mo_columns], axis=1)

df.head()


,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,...,0,1,2,3,4,5,6,7,8,9
846,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,WASHINGTON,OLIVE,...,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD)
19678,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,200 W AVENUE 26,NaN,...,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None
44979,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,SAWTELLE BL,OLYMPIC BL,...,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None
45032,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,ADAMS,GRAND,...,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision
45312,08/05/2024,835,77th Street,NaN,30,M,H,STREET,FLORENCE,GRAND,...,None,None,None,None,None,None,None,None,None,None


# Location Data

We will create a new column that contains the intersection of the address and cross street to get a better picture of high risk areas.

The code creates a function that combines the two, however important to note that intersections may show up twice in situations where the address and cross street are changed.

For example: 5th avenue & 6th street,
             6th street & 5th avenue are functionally the same.

We will avoid this by ordering the intersection in alphabetical order so the first street amongst the two will always take priority.
`


In [ ]:
# Create a function that will make the intersection
def intersection(i):
    streets = [i["Address"], i["Cross Street"]]
    streets = [x for x in streets if pd.notna(x)]
    return " , ".join(sorted(streets))

# removing white spaces from the two columns
df["Address"] = df["Address"].str.strip()
df["Cross Street"] = df["Cross Street"].str.strip()

# Apply the function to the DataFrame
df["Intersection"] = df.apply(intersection, axis=1)

df

,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,...,1,2,3,4,5,6,7,8,9,Intersection
846,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,WASHINGTON,OLIVE,...,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),"OLIVE , WASHINGTON"
19678,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,200 W AVENUE 26,NaN,...,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None,200 W AVENUE 26
44979,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,SAWTELLE BL,OLYMPIC BL,...,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None,"OLYMPIC BL , SAWTELLE ..."
45032,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,ADAMS,GRAND,...,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,"ADAMS , GRAND"
45312,08/05/2024,835,77th Street,NaN,30,M,H,STREET,FLORENCE,GRAND,...,None,None,None,None,None,None,None,None,None,"FLORENCE , GRAND"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619235,07/03/2024,2200,N Hollywood,3004 3028 3030 3101 4015,32,M,B,PARKING LOT,3800 BARHAM BL,NaN,...,T/C - (N) Non Injury,T/C - Hit and Run Misd,T/C - PCF (A) In the Narrative,T/C - North Hollywood,None,None,None,None,None,3800 BARHAM BL
619236,07/29/2024,1723,Mission,3004 3028 4026 3037 3101,99,M,O,STREET,WOODMAN AV,STRATHERN ST,...,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - At Intersection - No,T/C - PCF (A) In the Narrative,None,None,None,None,None,"STRATHERN ST , WOODMAN ..."
619237,07/25/2024,1525,Wilshire,3004 3026 3037 3038 4027 3035 3101 3401 3701,42,M,H,STREET,LA BREA AV,21ST ST,...,T/C - (C) Complaint of Injury,T/C - At Intersection - No,T/C - DUI Felony,T/C - West Traffic (WTD),T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Type of Collision,T/C - Movement Preceding Collision,None,"21ST ST , LA BREA ..."
619238,07/27/2024,1510,Devonshire,3101 3011 3028 4026 3030 3037,56,M,H,STREET,PARTHENIA,VALJEAN,...,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - Hit and Run Misd,T/C - At Intersection - No,None,None,None,None,"PARTHENIA , VALJEAN"


We will also be utilizing latitude and longitudinal data in our analysis, we will be splitting the data into two seperate columns.

In [ ]:
# Remove brackets
df["Location"]= df["Location"].str.strip("()")

# seperating the data into latitude and longitude
df[["Latitude", "Longitude"]] = df["Location"].str.split(",", expand=True)

# removing white spaces
df["Latitude"] = df["Latitude"].str.strip().astype(float)
df["Longitude"] = df["Longitude"].str.strip().astype(float)

df.drop("Location", axis=1, inplace=True)

df.head()

,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,...,3,4,5,6,7,8,9,Intersection,Latitude,Longitude
846,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,WASHINGTON,OLIVE,...,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),"OLIVE , WASHINGTON",34.0325,-118.2676
19678,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,200 W AVENUE 26,NaN,...,None,None,None,None,None,None,None,200 W AVENUE 26,34.0786,-118.2174
44979,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,SAWTELLE BL,OLYMPIC BL,...,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None,"OLYMPIC BL , SAWTELLE ...",34.0382,-118.4412
45032,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,ADAMS,GRAND,...,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,"ADAMS , GRAND",34.0265,-118.2723
45312,08/05/2024,835,77th Street,NaN,30,M,H,STREET,FLORENCE,GRAND,...,None,None,None,None,None,None,None,"FLORENCE , GRAND",33.9747,-118.2803


# Date, Time, Age, Sex, Descent

In [ ]:
#convert Date Occurred to datetime
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'], format='%m/%d/%Y')

In [ ]:
#convert Time Occurred to datetime
df['Time Occurred'] = pd.to_datetime(df['Time Occurred'].astype(str).str.zfill(4), format='%H%M').dt.strftime('%H:%M')

In [ ]:
#fill blanks with unknown/X for Victim Sex
for x in df.index:
  if df.loc[x, "Victim Sex"] == "F" or df.loc[x, "Victim Sex"] == "M":
    continue
  else:
    df.loc[x, "Victim Sex"] = "X"

In [ ]:
#fill blanks with unknown/X for Victim Descent
Descent_codes = ["A","B","C","D","F","G","H","I","J","K","L","O","P","S","U","V","W","Z"]

for x in df.index:
  if df.loc[x, "Victim Descent"] in Descent_codes:
    continue
  else:
    df.loc[x, "Victim Descent"] = "X"

In [ ]:
#recode age
print("Max Age - ",df['Victim Age'].max())
print("Min Age - ",df['Victim Age'].min())
print(pd.unique(df['Victim Age']))

Max Age -  99
Min Age -  10
<IntegerArray>
[  26,   31,   25,   30,   32, <NA>,   35,   24,   13,   23,   29,   28,   21,
   57,   51,   38,   33,   99,   22,   43,   20,   34,   72,   37,   17,   60,
   45,   63,   59,   27,   64,   55,   48,   46,   80,   65,   53,   44,   36,
   68,   49,   40,   19,   18,   67,   54,   61,   79,   69,   52,   39,   81,
   50,   73,   78,   41,   89,   74,   58,   42,   56,   47,   15,   62,   66,
   12,   70,   16,   82,   71,   76,   75,   10,   88,   77,   14,   85,   92,
   83,   91,   11,   96,   84,   86,   87,   90,   98,   94,   93,   95,   97]
Length: 91, dtype: Int64


In [ ]:
#checks for binning
print("<16 - ",df[df['Victim Age'] < 16].shape[0])
print("16-18 - ",df[(df['Victim Age'] >= 16) & (df['Victim Age'] < 18)].shape[0])
print("Missing  - ", df[df['Victim Age'].isna()].shape[0])

<16 -  317
16-18 -  294
Missing  -  3269


Teens between the age of 16-18 are permitted to drive if they hold a provincial driver's license in the state of California

# Cleaning age

In [ ]:
df= df.dropna(subset=['Victim Age'])

In [ ]:
#Creating bins for victim age
df.loc[:,'age_bin'] = pd.cut(df['Victim Age'], bins=[10, 16, 18, 30, 45, 60, np.inf],
                       labels=['Less than 16','16-18', '18-30', '30-45', '45-60', '60+'],
                       right=False, include_lowest=True)


<ipython-input-106-0005fc31f9d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'age_bin'] = pd.cut(df['Victim Age'], bins=[10, 16, 18, 30, 45, 60, np.inf],


In [ ]:
df.head()

,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,...,4,5,6,7,8,9,Intersection,Latitude,Longitude,age_bin
846,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,WASHINGTON,OLIVE,...,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),"OLIVE , WASHINGTON",34.0325,-118.2676,18-30
19678,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,200 W AVENUE 26,NaN,...,None,None,None,None,None,None,200 W AVENUE 26,34.0786,-118.2174,30-45
44979,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,SAWTELLE BL,OLYMPIC BL,...,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None,"OLYMPIC BL , SAWTELLE ...",34.0382,-118.4412,30-45
45032,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,ADAMS,GRAND,...,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,"ADAMS , GRAND",34.0265,-118.2723,18-30
45312,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,FLORENCE,GRAND,...,None,None,None,None,None,None,"FLORENCE , GRAND",33.9747,-118.2803,30-45


In [ ]:
print(df.shape)

(29793, 24)


Column Names will be adjusted after feature engineering to enhance readability for the rest of the project. We will also reoorder them so the seperated mo codes come after the original mo_code column

In [ ]:
# adjusting column names for easier interpretation for the rest of the project
df.columns = ['Date_Occurred', 'Time_Occurred', 'Area_Name', 'MO_Codes', 'Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Premise_Description', 'Address', 'Cross_Street', 'MO_Code01','MO_Code02','MO_Code03','MO_Code04','MO_Code05','MO_Code06','MO_Code07','MO_Code08','MO_Code09', 'MO_Code10', 'Intersection', 'Latitude', 'Longitude', 'Age_Group']

#Reordering columns

df = df.iloc[:, [0,1,2,3,10,11,12,13,14,15,16,17,18,19, 4, 5, 6, 7, 8, 9, 21, 22, 20, 23]]

In [ ]:
df = df.drop(columns=['Address', 'Cross_Street'])

df.head()

,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,MO_Code01,MO_Code02,MO_Code03,MO_Code04,MO_Code05,MO_Code06,...,MO_Code09,MO_Code10,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,Latitude,Longitude,Intersection,Age_Group
846,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,...,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),26,M,H,STREET,34.0325,-118.2676,"OLIVE , WASHINGTON",18-30
19678,2024-03-10,19:00,Hollenbeck,0605 3030 4024,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,...,None,None,31,M,C,STREET,34.0786,-118.2174,200 W AVENUE 26,30-45
44979,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,...,None,None,31,M,B,STREET,34.0382,-118.4412,"OLYMPIC BL , SAWTELLE ...",30-45
45032,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,...,T/C - Type of Collision,T/C - Movement Preceding Collision,25,M,O,SIDEWALK,34.0265,-118.2723,"ADAMS , GRAND",18-30
45312,2024-08-05,08:35,77th Street,NaN,None,None,None,None,None,None,...,None,None,30,M,H,STREET,33.9747,-118.2803,"FLORENCE , GRAND",30-45


In [ ]:
# rechecking the data type of each column
data_types = df.dtypes
data_types

,0
Date_Occurred,datetime64[ns]
Time_Occurred,object
Area_Name,object
MO_Codes,object
MO_Code01,object
MO_Code02,object
MO_Code03,object
MO_Code04,object
MO_Code05,object
MO_Code06,object


In [ ]:
#rename the final dataset
collision_data = df

collision_data.head()

,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,MO_Code01,MO_Code02,MO_Code03,MO_Code04,MO_Code05,MO_Code06,...,MO_Code09,MO_Code10,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,Latitude,Longitude,Intersection,Age_Group
846,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,...,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),26,M,H,STREET,34.0325,-118.2676,"OLIVE , WASHINGTON",18-30
19678,2024-03-10,19:00,Hollenbeck,0605 3030 4024,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,...,None,None,31,M,C,STREET,34.0786,-118.2174,200 W AVENUE 26,30-45
44979,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,...,None,None,31,M,B,STREET,34.0382,-118.4412,"OLYMPIC BL , SAWTELLE ...",30-45
45032,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,...,T/C - Type of Collision,T/C - Movement Preceding Collision,25,M,O,SIDEWALK,34.0265,-118.2723,"ADAMS , GRAND",18-30
45312,2024-08-05,08:35,77th Street,NaN,None,None,None,None,None,None,...,None,None,30,M,H,STREET,33.9747,-118.2803,"FLORENCE , GRAND",30-45


*** Day of Week, NEWWWWWW

In [ ]:
# Creates a new column with the day of the week according to the calendar
collision_data['Day'] = collision_data['Date_Occurred'].dt.day_name()

collision_data["Is_Weekend"] = collision_data["Day"].isin(["Saturday", "Sunday"])

In [ ]:
collision_data.head()

,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,MO_Code01,MO_Code02,MO_Code03,MO_Code04,MO_Code05,MO_Code06,...,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,Latitude,Longitude,Intersection,Age_Group,Day,Is_Weekend
846,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,...,26,M,H,STREET,34.0325,-118.2676,"OLIVE , WASHINGTON",18-30,Saturday,True
19678,2024-03-10,19:00,Hollenbeck,0605 3030 4024,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,...,31,M,C,STREET,34.0786,-118.2174,200 W AVENUE 26,30-45,Sunday,True
44979,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,...,31,M,B,STREET,34.0382,-118.4412,"OLYMPIC BL , SAWTELLE ...",30-45,Wednesday,False
45032,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,...,25,M,O,SIDEWALK,34.0265,-118.2723,"ADAMS , GRAND",18-30,Wednesday,False
45312,2024-08-05,08:35,77th Street,NaN,None,None,None,None,None,None,...,30,M,H,STREET,33.9747,-118.2803,"FLORENCE , GRAND",30-45,Monday,False


In [ ]:
# Exports to working file
collision_data.to_csv("collision_data.csv", index=False)